In [46]:
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer

### Load data

In [47]:
path = './data-20220511T090048Z-001/data/'
num_files = 6359

def load_data6359(load_path):
    """
    Input: Load path of data input
    Output: An list of strings correspond to data input

    """
    corpus = []
    for i in range(1, num_files + 1):
        with open(load_path + 'news' + str(i).zfill(5) + '.txt') as f:
            contents = f.read()
            corpus.append(contents)
    return corpus


def tf_idf(corpus):
    vectorizer = TfidfVectorizer(vocabulary=corpus)
    X = vectorizer.fit_transform(corpus)
    return X

def cosine_similarity(vec1 , vec2):
    return np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))
    
def vec(load_path):
    corpus = load_data6359(load_path)
    return tf_idf(corpus)

corpus = load_data6359(path)

### Preprocessing Functions
Tokenizing, removing stop words and stemming

In [48]:
import nltk

def get_tokenized_list(doc_text):
    """
    Return a list tokenized and of any text 
    """
    tokens = nltk.word_tokenize(doc_text)
    return tokens

def word_stemmer(token_list):
    """
    Return a list of word stemmed on tokenized words
    """
    ps = nltk.stem.PorterStemmer()
    stemmed = []
    for word in token_list:
        stemmed.append(ps.stem(word))
    return stemmed

def get_stopwords_vietnamesse(path):
    """
    Get data that contain stopwords in Vietnamese
    """
    with open(path) as f:
        return f.read().splitlines()

def remove_stopwords(doc_text):
    """
    Return a list of word after remove stopwords
    """
    stopwords_vn = get_stopwords_vietnamesse('stopword_vn.txt')
    removed_stopwords = []
    for word in doc_text:
        if word not in stopwords_vn:
            removed_stopwords.append(word)
    return removed_stopwords

def clean_word(text):
    """
    Return clean text after stemming and removing stopwords
    """
    token_list = get_tokenized_list(text)
    stemmed = word_stemmer(token_list)
    cleaned = remove_stopwords(stemmed)
    cleaned = ' '.join(cleaned)
    return cleaned


In [49]:
"""
Clean and save doc data

"""
# id = 0
# # clean doc data
# for text in corpus:
#     cleaned = clean_word(text)
#     id+=1 
#     file_out = open("./clean_doc_data/news" + str(id).zfill(5) + ".txt", "w")
#     file_out.write(cleaned)
    


'\nClean and save doc data\n\n'

In [50]:
data = load_data6359("./clean_doc_data/")

In [51]:
# Load and clean query
with open('query.txt') as f:
    query = f.read()
cleaned_query = clean_word(query)
data.append(cleaned_query)
print(len(data))

# TF-IDF
data_vec = tf_idf(data).toarray()

6360


ValueError: Duplicate term in vocabulary: "tàu điện cát linh - hà đông thể vận hành ? khăn hiện dự án cát linh - hà đông huy động chuyên gia việt nam hoàn thành đánh giá an toàn hệ thống . dự án đường sắt cát linh - hà đông khởi công 10/2011 , dự kiến vận hành quý ii/2019 , xác định ngài vận hành thức . hồi 6 , thủ tướng giao giao thông vận tải hoàn thiện , khai thác tuyến đường sắt đô thị cát linh - hà đông 2020 ; báo cáo phủ vướng mắc dự án , trình quốc hội hướng xử lý . tuyến đường chiều 13,05 km 12 ga 1 khu depot . tổng đầu tư ban đầu 8.770 tỷ đồng ( 553 triệu usd ) , chỉnh 18.002 tỷ đồng ( 868 triệu usd ) . nài , vốn vay oda trung quốc 13.867 tỷ đồng , vốn đối ứng 4.134 tỷ . nhiên , dự án nài tồn , vướng mắc liên quan thiết khu depot , đánh giá an toàn đoàn tàu , công tác vận hành toàn hệ thống quyết toán ... giao thông vận tải đạo giải quyết vướng mắc nài hoàn thiện hạng mục . giải quyết `` khăn '' nêu hoàn thành công tác đánh giá an toàn hệ thống . giao thông vận tải đề nghị ngo

In [ ]:
dtype = [('id', int), ('similarity', float)]
id_rel = []
for i in range(len(data_vec)-1):
    x = cosine_similarity(data_vec[-1], data_vec[i,:])
    if x > 0: 
        id_rel .append((i, x))
id_rel = np.array(id_rel, dtype=dtype)
id_rel

/tmp/ipykernel_36399/3779860881.py:24: RuntimeWarning: invalid value encountered in double_scalars
  return np.dot(vec1, vec2)/(np.linalg.norm(vec1)*np.linalg.norm(vec2))


array([], dtype=[('id', '<i8'), ('similarity', '<f8')])

In [ ]:
# Respond for query
num_responds = 10


respond= open("respond.txt", "w")
id_rel_sorted = np.sort(id_rel, order='similarity')[::-1]
respond.write(f"Top {num_responds} kết quả tốt nhất: \n\n")
for id in enumerate(id_rel_sorted[:num_responds]):
    respond.write(f"Rank: {id[0]+1} - Score: {id[1][1]} \n {corpus[id[1][0]]}\n")